Real time Tracking

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("/content/drive/My Drive/ModelBVA.pt")

# Open the video file
video_path = "/content/drive/My Drive/Big Vision Assignment/Output on vidoes/output_video_1.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Break if the video ends

    # Use YOLOv8's built-in tracking function with BoT-SORT
    # results = model.track(frame, persist=True, tracker="bytetrack.yaml")
    results = model.track(frame, persist=True, tracker="bytetrack.yaml", conf=0.35, iou=0.8)

    # Draw bounding boxes with tracking IDs
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            track_id = int(box.id[0]) if box.id is not None else -1  # Tracking ID
            conf = box.conf[0].item()  # Confidence score
            label = int(box.cls[0])  # Class label

            # Draw rectangle and ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID {track_id}: {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the frame with detections
    cv2.imshow("YOLOv8 + Byte-track Tracking", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Getting MOTA output as txt


In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("/Users/aayushjain/Downloads/sportsmot_publish/Final models/runs/detect/train4/weights/best.pt")

# Open the video file
video_path = "/Users/aayushjain/Downloads/sportsmot_publish/output_video_MOTA.mp4"
cap = cv2.VideoCapture(video_path)

# Define output file for tracking results
output_txt = "/Users/aayushjain/Downloads/sportsmot_publish/tracker_output.txt"
f = open(output_txt, "w")

frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    # Run YOLOv8 + ByteTrack
    results = model.track(frame, persist=True, tracker="bytetrack.yaml", conf=0.5, iou=0.7)

    # Ensure detections exist
    if results and len(results) > 0:
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
                track_id = int(box.id[0]) if box.id is not None else -1  # Tracking ID
                conf = box.conf[0].item()  # Confidence score

                # Save to file (Format: frame_id, track_id, x1, y1, x2, y2, confidence)
                f.write(f"{frame_id},{track_id},{x1},{y1},{x2},{y2},{conf:.2f}\n")

    frame_id += 1

cap.release()
f.close()
print(f"✅ Tracking data saved at: {output_txt}")

Check MOTA

In [ ]:
import numpy as np

# Load data
gt_data = np.loadtxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/gt_fixed.txt", delimiter=",")
track_data = np.loadtxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/tracker_output.txt", delimiter=",")

# Check unique frames
print("Unique frames in GT:", np.unique(gt_data[:, 0]))
print("Unique frames in Tracker Output:", np.unique(track_data[:, 0]))

gt fix

In [ ]:
import numpy as np

# Load GT
gt_data = np.loadtxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/gt.txt", delimiter=",")

# Convert bbox format
gt_data[:, 4] += gt_data[:, 2]  # x2 = x1 + width
gt_data[:, 5] += gt_data[:, 3]  # y2 = y1 + height

# Save fixed GT
np.savetxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/gt_fixed.txt", gt_data, delimiter=",", fmt="%.2f")

print("✅ Ground truth converted to (x1, y1, x2, y2) format and saved!")

tracker fixed

In [ ]:
import numpy as np

# Load tracker output
track_data = np.loadtxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/tracker_output.txt", delimiter=",")

# Increment frame IDs by 1
track_data[:, 0] += 1

# Save back
np.savetxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/tracker_output_fixed.txt", track_data, delimiter=",", fmt="%.2f")

print("✅ Fixed tracker_output.txt saved as tracker_output_fixed.txt")

Fix MOTA

In [ ]:
import motmetrics as mm
import numpy as np

# Initialize MOT accumulator
acc = mm.MOTAccumulator(auto_id=True)

# Load ground truth & tracker predictions (format: frame_id, obj_id, x1, y1, x2, y2)
gt_data = np.loadtxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/gt_fixed.txt", delimiter=",")  # Replace with your GT file
track_data = np.loadtxt("/Users/aayushjain/codes/projects/company assignements/Big Vision/tracker_output_fixed.txt", delimiter=",")  # Replace with ByteTrack results

# Process frame by frame
for frame in np.unique(gt_data[:, 0]):  # Loop through all frames
    gt_boxes = gt_data[gt_data[:, 0] == frame, 2:6]  # Extract GT bbox
    track_boxes = track_data[track_data[:, 0] == frame, 2:6]  # Extract detected bbox
    gt_ids = gt_data[gt_data[:, 0] == frame, 1].astype(int)  # GT IDs
    track_ids = track_data[track_data[:, 0] == frame, 1].astype(int)  # Tracked IDs

    # Compute IoU (Intersection Over Union) for matching GT with predicted tracks
    distances = mm.distances.iou_matrix(gt_boxes, track_boxes, max_iou=0.5)

    # Update MOT accumulator with matches
    acc.update(gt_ids, track_ids, distances)

# Compute metrics
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=["num_misses", "num_false_positives", "num_switches", "mota"], name="MOTA Results")
print(summary)

Print Summary

In [ ]:
import motmetrics as mm
import numpy as np

# Load Ground Truth (GT) and Tracker Output
gt_file = "/Users/aayushjain/codes/projects/company assignements/Big Vision/gt_fixed.txt"
tracker_file = "/Users/aayushjain/codes/projects/company assignements/Big Vision/tracker_output_fixed.txt"

gt_data = np.loadtxt(gt_file, delimiter=",")  # GT: (frame_id, obj_id, x1, y1, x2, y2)
track_data = np.loadtxt(tracker_file, delimiter=",")  # Tracker: (frame_id, track_id, x1, y1, x2, y2, conf)

# Initialize MOT accumulator
acc = mm.MOTAccumulator(auto_id=True)

# Process frame by frame
for frame in np.unique(gt_data[:, 0]):  # Loop through all frames
    gt_boxes = gt_data[gt_data[:, 0] == frame, 2:6]  # GT bounding boxes
    track_boxes = track_data[track_data[:, 0] == frame, 2:6]  # Tracker bounding boxes
    gt_ids = gt_data[gt_data[:, 0] == frame, 1].astype(int)  # GT IDs
    track_ids = track_data[track_data[:, 0] == frame, 1].astype(int)  # Tracked IDs

    # Compute IoU (Intersection Over Union) for matching
    distances = mm.distances.iou_matrix(gt_boxes, track_boxes, max_iou=0.5)

    # Update MOT accumulator
    acc.update(gt_ids, track_ids, distances)

# Compute MOTA, IDF1, and MOTP
mh = mm.metrics.create()
metrics = ["idf1", "mota", "motp"]
summary = mh.compute(acc, metrics=metrics, name="Tracking Performance")

# Print results
print(summary)